# Week 4: Milestone 1 – Smart Tracking System Blockchain Ledger (Draft)

Follow these steps to verify Python↔Ganache↔Smart Contract connection.

1. Setup and Connection

1. **Open Jupyter Notebook**  
2. **Verify Ganache** is running (Desktop App “Quickstart Ethereum” or `ganache-cli`).  
3. **Check RPC port** (default is `7545` in Ganache Desktop).

Below we’ll connect Python → Ganache → our IoTDataStorage contract.

In [2]:
# Cell 1: imports & connection
from web3 import Web3
import json, os

# 1️⃣ Point to your Ganache RPC (update port if needed)
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 2️⃣ Test connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


2. Load Contract ABI and Address

🔗 Now we load the **ABI** exported (in `/artifacts/IoTDataStorageABI.json`) and the **deployed address** from Remix.

In [3]:
# Cell 2: load ABI + contract
abi_path = os.path.join("..", "artifacts", "IoTDataStorageABI.json")
with open(abi_path) as f:
    abi = json.load(f)

# Replace with actual deployed address - convert to checksum format
contract_address_raw = "0x03c92d872555d667bdff0e54beb3557f1c4a154e"
contract_address = Web3.to_checksum_address(contract_address_raw)

contract = web3.eth.contract(address=contract_address, abi=abi)

# Use the Ganache account that has a balance
web3.eth.default_account = web3.eth.accounts[1]

print(f"▶️ Using account {web3.eth.default_account}")
print(f"✅ Loaded contract at {contract_address}")

# Optional: Test the connection
try:
    total_records = contract.functions.getTotalRecords().call()
    print(f"📊 Current total records: {total_records}")
    print("🎉 Contract connection successful!")
except Exception as e:
    print(f"❌ Error testing contract connection: {e}")

▶️ Using account 0x805F8eb842DcaD1Fc999eE8257Cd0C99BFC9d13d
✅ Loaded contract at 0x03C92d872555D667bdFf0e54BEB3557f1C4a154E
📊 Current total records: 0
🎉 Contract connection successful!


3. Read-Only Call: getTotalRecords()

Let’s check that our contract is responding:

In [4]:
# Cell 3: read-only call
total = contract.functions.getTotalRecords().call()
print(f"🔢 Total stored entries on‐chain: {total}")

🔢 Total stored entries on‐chain: 0


4. Write a dummy entry

Store one dummy IoT record (TEST001) to prove transactions work.

In [5]:
# Cell 4: store dummy entry

# First, make sure we have a valid account
print("Available accounts:")
for i, account in enumerate(web3.eth.accounts):
    balance = web3.eth.get_balance(account)
    print(f"  [{i}] {account} - Balance: {web3.from_wei(balance, 'ether')} ETH")

# Set the account explicitly (use account[0] which is usually the deployer)
sender_account = web3.eth.accounts[0]  # or web3.eth.accounts[1] if you prefer
print(f"\n▶️ Using account: {sender_account}")

# Store dummy data
tx = contract.functions.storeData(
    "TEST003",      # dummy ID
    "New York",  # Location
    "22.5°C"       # Temperature
).transact({
    "from": sender_account,  # Use explicit account instead of default_account
    "gas": 200_000,
    "gasPrice": web3.to_wei("1", "gwei")
})

print(f"📝 Transaction sent: {tx.hex()}")

# Wait for transaction to be mined
receipt = web3.eth.wait_for_transaction_receipt(tx)
print(f"✅ Dummy data tx mined: {receipt.transactionHash.hex()}")
print(f"📊 Gas used: {receipt.gasUsed}")

# Verify the data was stored
try:
    total_records = contract.functions.getTotalRecords().call()
    print(f"🎯 Total records after storing: {total_records}")
    
    if total_records > 0:
        # Get the latest record
        latest_record = contract.functions.getRecord(total_records - 1).call()
        print(f"📋 Latest record: {latest_record}")
except Exception as e:
    print(f"❌ Error reading stored data: {e}")

Available accounts:
  [0] 0x23e50c0f38de3D94596cd254253401Fb32365b85 - Balance: 99.9989576768125 ETH
  [1] 0x805F8eb842DcaD1Fc999eE8257Cd0C99BFC9d13d - Balance: 100 ETH
  [2] 0x44B1c26097d7f471Ce0ACC5764e733dC0501523f - Balance: 100 ETH
  [3] 0x869c81855A9AAE18C9Ba055C3712B175b0750CAB - Balance: 100 ETH
  [4] 0xd1b2EACFc75dEbEbefA1d2F35CAe5Fe9F6A62C4B - Balance: 100 ETH
  [5] 0x47D0B34D7DFFf27365F192FE97AbFd0edE1C30a5 - Balance: 100 ETH
  [6] 0x5E3B9DB6E87D1f130BAFb137d375E618a1b0eE54 - Balance: 100 ETH
  [7] 0xbD2B3b60C1B7D98FabDC1E504e2b4460A7E7756C - Balance: 100 ETH
  [8] 0x3e19d0a1E7639e581a324f6dAE5424691333C554 - Balance: 100 ETH
  [9] 0x7695c219fd0261b748D8524a9390515a87fb69e2 - Balance: 100 ETH

▶️ Using account: 0x23e50c0f38de3D94596cd254253401Fb32365b85
📝 Transaction sent: 30cc7e91650c71673644850d9ae86657d090a580c200fbe967e54bc466e482d6
✅ Dummy data tx mined: 30cc7e91650c71673644850d9ae86657d090a580c200fbe967e54bc466e482d6
📊 Gas used: 144765
🎯 Total records after storing: 1


5. Verify it went through

After mining, call again to see the updated count and fetch the index.

In [8]:
# Cell 5: verify storage
new_total = contract.functions.getTotalRecords().call()
print(f"🔢 New total entries: {new_total}")

first = contract.functions.getRecord(0).call()
print("📦 First stored record:", first)

🔢 New total entries: 1
📦 First stored record: [1749516561, 'TEST003', 'New York', '22.5°C']


In [9]:
# Cell 6: Adding New Iot Data using our Output 1(logistics_data.json) on the Blockchain

import pandas as pd

df = pd.read_csv("../data/logistics_data.csv")

sender_account = web3.eth.accounts[0]
print(f"🔐 Using account: {sender_account}")

for i, row in df.iterrows():
    try:
        # Convert entire row to a single string
        row_string = "|".join(str(row[col]) for col in df.columns)

        tx = contract.functions.storeData(
            str(row['shipment_id']),  # Key or ID
            "FullRow",                # Data type or label
            row_string                # Entire row as single string
        ).transact({
            "from": sender_account,
            "gas": 300_000,
            "gasPrice": web3.to_wei("1", "gwei")
        })

        receipt = web3.eth.wait_for_transaction_receipt(tx)
        print(f"[{i+1}/{len(df)}] ✅ Stored full row for: {row['shipment_id']} | Tx: {tx.hex()}")

    except Exception as e:
        print(f"[{i+1}/{len(df)}] ❌ Error storing full row for {row['shipment_id']}: {e}")


🔐 Using account: 0x23e50c0f38de3D94596cd254253401Fb32365b85
[1/100] ✅ Stored full row for: SHIP7734 | Tx: 27dad99b1a1f9feaceb66a81f44d47b11f62423f8c5821054af2031116afec51
[2/100] ✅ Stored full row for: SHIP4309 | Tx: f1562ae66f3ebbbc4d9258e87160b6f36d50993e05f172c441d96bc23ad343ca
[3/100] ✅ Stored full row for: SHIP1802 | Tx: 34bbb7974ba20cecf2f6ee1723bb59cc2a25ff831f494e1853bdbac76a943577
[4/100] ✅ Stored full row for: SHIP9967 | Tx: 31edd75d305930057c0471c0082c06bb2de1ae1c32aa1d72877e494382e5fe82
[5/100] ✅ Stored full row for: SHIP3088 | Tx: 76e795478da2845d0ea2b89868a718a34dd8d24cf18994bce8589274cc177646
[6/100] ✅ Stored full row for: SHIP9457 | Tx: 4ff07ac8be3215627ab7bd8290f323f312484dc1754392e5b430e903e37b9a71
[7/100] ✅ Stored full row for: SHIP7133 | Tx: 24bea3c0ccd7c110c11c9e5fcab6c01d54ba0e2ef32a9607480feedf4ff0133b
[8/100] ✅ Stored full row for: SHIP5276 | Tx: 1cd2bd4e9de9a004976a210843d19e6a54860fdb1be7030f2f13a22e783c4adb
[9/100] ✅ Stored full row for: SHIP9411 | Tx: 88928f

In [18]:
# Read first row from CSV

# For this example we use the guide from camu which is the sampler data
first_record = contract.functions.getRecord(0).call()
print("CSV First Record (concatenated):")
# TEST003
# New York
# 22.5°C
print(f"  Package Name: {first_record[0]}")
print(f"  Location: {first_record[2]}")
print(f"  Temperature: {first_record[3]}")

# In this case we use our generated logistics data
# Fetch last record from blockchain
last_record = contract.functions.getRecord(100).call()

print("\nBlockchain Last Record:")
print(f"  Shipment ID: {last_record[0]}")
print(f"  Data Type: {last_record[1]}")
print(f"  Full Row Data: {last_record[2]}")


CSV First Record (concatenated):
  Package Name: 1749516561
  Location: New York
  Temperature: 22.5°C

Blockchain Last Record:
  Shipment ID: 1749516919
  Data Type: SHIP5841
  Full Row Data: FullRow


# Data Retrieval and Processing

In [16]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 101


In [26]:
import pandas as pd


# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "device_id": record[1],
        "data_type": record[2],
        "data_value": record[3]
    })


# Convert to a DataFrame
df = pd.DataFrame(data)


# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")


# Display first few records
print(df.head())

            timestamp device_id data_type  \
0 2025-06-10 00:49:21   TEST003  New York   
1 2025-06-10 00:55:15  SHIP7734   FullRow   
2 2025-06-10 00:55:15  SHIP4309   FullRow   
3 2025-06-10 00:55:15  SHIP1802   FullRow   
4 2025-06-10 00:55:15  SHIP9967   FullRow   

                                          data_value  
0                                             22.5°C  
1  2025-05-18 15:46:54|2025-05-18 11:46:54|2025-0...  
2  2025-05-18 09:41:54|2025-05-18 06:41:54|2025-0...  
3  2025-05-18 13:45:54|2025-05-18 10:45:54|2025-0...  
4  2025-05-18 05:57:54|2025-05-18 01:57:54|2025-0...  


In [24]:

import numpy as np


# Extract numeric values from 'data_value' where applicable
df["numeric_value"] = df["data_value"].str.extract(r'(\d+\.?\d*)').astype(float)


# Handle missing values (if any)
df.fillna(0, inplace=True)


# Display cleaned data
print(df.head())

            timestamp device_id data_type  \
0 2025-06-10 00:49:21   TEST003  New York   
1 2025-06-10 00:55:15  SHIP7734   FullRow   
2 2025-06-10 00:55:15  SHIP4309   FullRow   
3 2025-06-10 00:55:15  SHIP1802   FullRow   
4 2025-06-10 00:55:15  SHIP9967   FullRow   

                                          data_value  numeric_value  
0                                             22.5°C           22.5  
1  2025-05-18 15:46:54|2025-05-18 11:46:54|2025-0...         2025.0  
2  2025-05-18 09:41:54|2025-05-18 06:41:54|2025-0...         2025.0  
3  2025-05-18 13:45:54|2025-05-18 10:45:54|2025-0...         2025.0  
4  2025-05-18 05:57:54|2025-05-18 01:57:54|2025-0...         2025.0  
